In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# The goal is to check if summary-features are returned with grouping requests.

In [3]:
from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.637.17",
)
# Start a docker container and deploy the application package
client = vespa_docker.deploy(
    application_package=vap.demo_application_package(),
)

Application is up!
Finished deployment.


In [4]:
# Create and feed 1 dummy doc
docs = [
    {
        'id': f'{1}',
        'fields': {
            'id': 1,
        }
    }
]

client.feed_iterable(docs, schema="doc", namespace="doc", callback=vap.feed_callback)

In [7]:
client.query(body={
    'yql': 'select * from sources * where true limit 1',
    'ranking.profile': 'fields'
}).json

{'root': {'children': [{'fields': {'documentid': 'id:doc:doc::1',
     'matchfeatures': {'id': 1.0},
     'sddocname': 'doc',
     'summaryfeatures': {'id': 1.0, 'vespa.summaryFeatures.cached': 0.0}},
    'id': 'id:doc:doc::1',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 1},
  'id': 'toplevel',
  'relevance': 1.0}}

In [9]:
client.query(body={
    'yql': '''
           select *
           from sources *
           where true
               limit 1
               | all(group(id) each(each( output(summary(default)))))
           ''',
    'ranking.profile': 'fields'
}).json

{'root': {'children': [{'children': [{'children': [{'children': [{'children': [{'fields': {'documentid': 'id:doc:doc::1',
             'sddocname': 'doc',
             'summaryfeatures': {'id': 1.0,
              'vespa.summaryFeatures.cached': 0.0}},
            'id': 'id:doc:doc::1',
            'relevance': 0.0,
            'source': 'test_content'}],
          'id': 'hitlist:hits',
          'label': 'hits',
          'relevance': 1.0}],
        'id': 'group:long:1',
        'relevance': 0.0,
        'value': '1'}],
      'id': 'grouplist:id',
      'label': 'id',
      'relevance': 1.0}],
    'continuation': {'this': ''},
    'id': 'group:root:0',
    'relevance': 1.0},
   {'fields': {'documentid': 'id:doc:doc::1',
     'matchfeatures': {'id': 1.0},
     'sddocname': 'doc',
     'summaryfeatures': {'id': 1.0, 'vespa.summaryFeatures.cached': 0.0}},
    'id': 'id:doc:doc::1',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 1,
   

In [10]:
a = [{'a': 1}, {'a': 2}]

In [15]:
list(map(lambda index, fields: {'id': index, 'fields': fields}, enumerate(a)))

TypeError: <lambda>() missing 1 required positional argument: 'fields'

In [20]:

list(map(lambda doc: {'index': doc[0], 'elem': doc[1]}, enumerate(['a', 'b'])))

[{'index': 0, 'elem': 'a'}, {'index': 1, 'elem': 'b'}]

In [21]:
vap.feed(client, docs=[{'id': 2}, {'id': 3}])

In [22]:
client.query(body={
    'yql': 'select * from sources * where true',
}).json

{'root': {'children': [{'fields': {'documentid': 'id:doc:doc::1',
     'sddocname': 'doc'},
    'id': 'id:doc:doc::1',
    'relevance': 0.0,
    'source': 'test_content'},
   {'fields': {'documentid': 'id:doc:doc::0', 'sddocname': 'doc'},
    'id': 'id:doc:doc::0',
    'relevance': 0.0,
    'source': 'test_content'}],
  'coverage': {'coverage': 100,
   'documents': 2,
   'full': True,
   'nodes': 1,
   'results': 1,
   'resultsFull': 1},
  'fields': {'totalCount': 2},
  'id': 'toplevel',
  'relevance': 1.0}}